<div class="alert alert-block alert-success"> <h3 style="color: #155724; font-weight: bold;">🚀 Before You Begin</h3>
To visualize the data in this demonstration, we will be using the Mapbox SDK. If you don’t already have a Mapbox account or didn’t open the notebook with your access token, please follow these steps:

1. Create a Mapbox Account
Go to Mapbox and sign up for a free account.

Once registered, navigate to your account dashboard and generate an access token. The token will look something like this: pk.eyJ....

2. Set Up Your Environment
To ensure the notebook works correctly, open it through your terminal or console and pass your Mapbox access token as an environment variable:

<span style="background: #000000; padding: 2px 5px; border-radius: 3px; border: 1px solid #444; font-family: monospace; color: #ffffff;"> kamu notebook -e MAPBOX_ACCESS_TOKEN=&lt;your_mapbox_token&gt; </span>

Replace <span style="background: #000000; padding: 2px 5px; border-radius: 3px; border: 1px solid #444; font-family: monospace; color: #ffffff;">&lt;your_mapbox_token&gt;</span> with the token you obtained from Mapbox.

3. Start Exploring
Once the notebook is running, you’re all set to dive into the world of GIS visualization! 🗺️

</div>

<div class="alert alert-block alert-success">
The first few steps are for loading and testing the data. 
    Green text provides a description of what is happening, while blue text is likely intended for the blog.
</div>

In [ ]:
%load_ext kamu

In [ ]:
import kamu
con = kamu.connect()

In [ ]:
%%sql -o historic_fires
SELECT * FROM 'spatial.fires' limit 10

<div class="alert alert-block alert-success">
The following code demonstrates the average and largest fire sizes for each decade, illustrating the overall trends.
</div>

In [ ]:
%%sql
SELECT 
    FLOOR(CAST("FIRE_DATE" AS INTEGER)/10000/10)*10 as decade,
    COUNT(*) as num_fires,
    AVG(area_sqm) as avg_area_sqm,
    MAX(area_sqm) as largest_fire_sqm
FROM 'spatial.fires'
GROUP BY FLOOR(CAST("FIRE_DATE" AS INTEGER)/10000/10)*10
ORDER BY decade

<div class="alert alert-block alert-info">
FFrom our earliest records of fire patterns in British Columbia, the data tells a compelling story of change and adaptation. Pre-1910 records show just three documented fires, though these early blazes could reach up to 21.9 million square meters. By the 1910s, monitoring improved significantly, with 263 recorded fires and the largest reaching 168.9 million square meters - a pattern influenced by the industrial expansion of railways, logging, and settlement.
The 1920s and 1930s marked a period of intense fire activity, with approximately 3,900 fires per decade. Individual fires during this time averaged 5.89 and 4.64 million square meters respectively, though the largest could reach up to 684.7 million square meters. The 1938 Bloedel Fire demonstrated this destructive potential, consuming an entire community and prompting significant changes in fire management approaches.
A notable shift occurred in the 1940s and 1950s. While fire frequency decreased to around 1,500 incidents per decade, their intensity grew considerably. The 1950 Chinchaga Fire, known locally as the Wisp Fire, reached an unprecedented 2.44 billion square meters, creating the remarkable Great Smoke Pall that darkened skies across eastern North America.
Fire patterns maintained relative stability through the following decades, averaging 1,400-1,570 fires per decade until the 1990s, when numbers dropped to 951. However, the landscape changed dramatically with the new millennium. The 2000s recorded 2,527 fires averaging 5.14 million square meters, while the 2010s saw 3,246 fires with an average size of 11.88 million square meters - a significant increase influenced by the Mountain Pine Beetle outbreak and changing climate conditions.
The 2020s have already demonstrated unprecedented fire behavior. Despite recording fewer fires (1,625), the average size has increased dramatically to 28.89 million square meters. The largest fire of this period reached an astounding 7.69 billion square meters in the Prince George and Vanderhoof areas, setting new records and highlighting the growing challenges of fire management in our changing climate.
This progression, supported by detailed data analysis, illustrates the evolving nature of wildfire behavior in British Columbia and underscores the importance of adaptive management strategies as we face future challenges.
</div>

In [ ]:
import plotly.graph_objs as go
import pandas as pd

# Corrected data
data = {
    'decade': ['Pre-1910', '1910', '1920', '1930', '1940', '1950', 
               '1960', '1970', '1980', '1990', '2000', '2010', '2020', '2030'],
    'num_fires': [3, 263, 3935, 3946, 1483, 1528, 
                  1570, 1502, 1410, 951, 2527, 3246, 1625, None],  # Corrected values
    'largest_fire_sqm': [2.186581e+07, 1.689352e+08, 5.828499e+08, 6.846574e+08, 
                        1.324948e+09, 2.440266e+09, 5.106355e+08, 9.889928e+08, 
                        1.666985e+09, 2.376718e+08, 6.774347e+08, 5.208852e+09, 
                        7.689150e+09, None]
}

df = pd.DataFrame(data)

def create_enhanced_fire_visualization():
    fig = go.Figure()

    # Number of Fires trace
    fig.add_trace(go.Scatter(
        x=list(range(len(df['decade']))),
        y=df['num_fires'],
        mode='lines+markers',
        name='Number of Fires',
        yaxis='y1',
        line=dict(
            color='rgba(66, 135, 245, 0.8)',
            width=2
        ),
        marker=dict(
            size=8,
            color='rgba(66, 135, 245, 0.8)'
        ),
        fill='tonexty',
        fillcolor='rgba(66, 135, 245, 0.1)'
    ))

    # Largest Fire Area trace
    fig.add_trace(go.Scatter(
        x=list(range(len(df['decade']))),
        y=df['largest_fire_sqm'],
        mode='lines+markers',
        name='Largest Fire Area (sq meters)',
        yaxis='y2',
        line=dict(
            color='rgba(255, 89, 94, 0.8)',
            width=2
        ),
        marker=dict(
            size=8,
            color='rgba(255, 89, 94, 0.8)'
        ),
        fill='tonexty',
        fillcolor='rgba(255, 89, 94, 0.1)'
    ))

    # Add significant fires with stars
    historical_events = [
        {'decade': '1930', 'event': 'Bloedel Fire (1938)', 'size': 6.846574e+08, 'index': 3, 'position': 'left'},
        {'decade': '1950', 'event': 'Chinchaga Fire (1950)', 'size': 2.440266e+09, 'index': 5, 'position': 'left'},
        {'decade': '2010', 'event': 'Plateau Complex (2017)', 'size': 5.208852e+09, 'index': 11, 'position': 'left'},
        {'decade': '2020', 'event': 'Record Season (2023)', 'size': 7.689150e+09, 'index': 12, 'position': 'left'}
    ]

    for event in historical_events:
        fig.add_trace(go.Scatter(
            x=[event['index']],
            y=[event['size']],
            mode='markers+text',
            name=event['event'],
            yaxis='y2',
            marker=dict(
                symbol='star',
                size=12,
                color='red'
            ),
            text=event['event'],
            textposition="top left",
            showlegend=True
        ))

    # Add time period shapes
    shapes = [
        dict(
            type="rect",
            xref="x",
            yref="paper",
            x0=0,
            x1=3,
            y0=0,
            y1=1,
            fillcolor="rgba(128, 128, 128, 0.2)",
            layer="below",
            line_width=0,
        ),
        dict(
            type="rect",
            xref="x",
            yref="paper",
            x0=10,
            x1=11,
            y0=0,
            y1=1,
            fillcolor="rgba(255, 165, 0, 0.2)",
            layer="below",
            line_width=0,
        )
    ]

    # Add annotations for time periods
    annotations = [
        dict(
            x=1.5,
            y=1.05,
            xref="x",
            yref="paper",
            text="Before Reforesting Project of BC",
            showarrow=False,
            font=dict(size=12),
            textangle=0
        ),
        dict(
            x=10.5,
            y=1.05,
            xref="x",
            yref="paper",
            text="Mountain Pine Beetle Outbreak",
            showarrow=False,
            font=dict(size=12),
            textangle=0
        )
    ]

    # Update layout
    fig.update_layout(
        title=dict(
            text='British Columbia Wildfires (1910s-2020s)',
            font=dict(size=24),
            x=0.4,
            y=0.95
        ),
        plot_bgcolor='white',
        paper_bgcolor='white',
        xaxis=dict(
            title='Decade',
            showgrid=True,
            gridwidth=1,
            gridcolor='rgba(189, 195, 199, 0.2)',
            tickmode='array',
            ticktext=df['decade'].tolist(),
            tickvals=list(range(len(df['decade']))),
            tickangle=0
        ),
        yaxis=dict(
            title='Number of Fires',
            titlefont=dict(color='rgba(66, 135, 245, 0.8)'),
            tickfont=dict(color='rgba(66, 135, 245, 0.8)'),
            showgrid=True,
            gridwidth=1,
            gridcolor='rgba(189, 195, 199, 0.2)',
            range=[0, 8000]
        ),
        yaxis2=dict(
            title='Fire Area (square meters)',
            titlefont=dict(color='rgba(255, 89, 94, 0.8)'),
            tickfont=dict(color='rgba(255, 89, 94, 0.8)'),
            overlaying='y',
            side='right',
            range=[0, 1e10],
            showgrid=False
        ),
        legend=dict(
            x=1.05,
            y=1,
            bgcolor='rgba(255, 255, 255, 0.8)'
        ),
        height=600,
        width=900,
        margin=dict(
            t=100,
            r=150,
            l=80,
            b=50
        ),
        showlegend=True,
        hovermode='x unified',
        shapes=shapes,
        annotations=annotations
    )

    return fig

# Create and display visualization
fig = create_enhanced_fire_visualization()
fig.show()

<div class="alert alert-block alert-success">
This analysis reveals a potential bias in the historical fire records, as most documented fires were larger than 100,000 square meters. This suggests that smaller fires may have been systematically underreported, skewing the data toward larger fire events.
</div>

In [ ]:
%%sql
SELECT 
    CASE 
        WHEN area_sqm < 1000 THEN 'Small (<1000 sq m)'
        WHEN area_sqm < 10000 THEN 'Medium (1000-10000 sq m)'
        WHEN area_sqm < 100000 THEN 'Large (10000-100000 sq m)'
        ELSE 'Very Large (>100000 sq m)'
    END as size_category,
    COUNT(*) as number_of_fires,
    ROUND(AVG(area_sqm), 2) as avg_size
FROM 'spatial.fires'
GROUP BY size_category
ORDER BY avg_size

<div class="alert alert-block alert-info">
Historical data from British Columbia's fire records reveals telling patterns in both fire frequency and size distribution. Of the documented fires, 18,136 reached the "Very Large" category, exceeding 100,000 square meters and averaging 11.08 million square meters per incident. Large fires (10,000-100,000 square meters) account for 4,851 events, with an average size of 40,694 square meters.
The data shows a notable skew toward larger fires, with fewer recordings of smaller incidents. Medium-sized fires (1,000-10,000 square meters) account for 742 documented cases averaging 5,492 square meters, while only 260 small fires (under 1,000 square meters) were recorded, averaging 278 square meters each.
This distribution likely reflects historical reporting practices, where larger, more impactful fires received thorough documentation while smaller incidents may have gone unrecorded. However, recent decades show a clear trend toward more frequent and intense fires, driven by several key factors: the extensive dead timber left by the Mountain Pine Beetle outbreak, prolonged drought conditions linked to climate change, and increased lightning activity.
This changing dynamic of fire behavior and frequency presents growing challenges for modern fire management strategies, particularly as environmental conditions continue to evolve.
</div>

<div class="alert alert-block alert-success">
These next steps demonstrate the differences between forest fires caused by people and those caused by lightning. People caused more fires in the earlier decades, while lightning has been the primary cause in recent decades. Although more fires were caused by people over this time, the fires caused by lightning resulted in the burning of larger areas.
</div>

In [ ]:
%%sql
WITH total_stats AS (
    SELECT 
        COUNT(*) as total_fires,
        SUM(area_sqm) as total_area
    FROM 'spatial.fires'
)
SELECT 
    fire_cause,
    COUNT(*) as num_fires,
    ROUND(AVG(area_sqm), 2) as avg_size,
    ROUND(MAX(area_sqm), 2) as largest_fire,
    ROUND(SUM(area_sqm), 2) as total_area_burned,
    ROUND(COUNT(*) * 100.0 / total_stats.total_fires, 2) as percentage_of_fires,
    ROUND(SUM(area_sqm) * 100.0 / total_stats.total_area, 2) as percentage_of_area
FROM 'spatial.fires'
CROSS JOIN total_stats
GROUP BY fire_cause, total_stats.total_fires, total_stats.total_area
ORDER BY 
    num_fires DESC,
    total_area_burned DESC;

In [ ]:
%%sql
SELECT 
    FLOOR(CAST("FIRE_DATE" AS INTEGER)/10000/10)*10 as decade,
    fire_cause,
    COUNT(*) as frequency
FROM 'spatial.fires'
GROUP BY FLOOR(CAST("FIRE_DATE" AS INTEGER)/10000/10)*10, fire_cause
ORDER BY decade, frequency DESC

In [ ]:
import plotly.express as px

# Create the dataframe from the original data
data = {
    'decade': ['NaN', 'NaN', '1910', '1910', '1920', '1920', '1930', '1930', 
               '1940', '1940', '1950', '1950', '1960', '1960', '1970', '1970',
               '1980', '1980', '1990', '1990', '2000', '2000', '2010', '2010',
               '2010', '2020', '2020', '2020'],
    'fire_cause': ['Lightning', 'Unknown', 'Person', 'Lightning', 'Person', 'Lightning',
                   'Person', 'Lightning', 'Person', 'Lightning', 'Person', 'Lightning',
                   'Person', 'Lightning', 'Person', 'Lightning', 'Person', 'Lightning',
                   'Person', 'Lightning', 'Person', 'Lightning', 'Lightning', 'Person',
                   'Unknown', 'Lightning', 'Person', 'Unknown'],
    'frequency': [2, 1, 220, 43, 2959, 976, 3175, 771, 996, 487, 1103, 425,
                 1040, 530, 1114, 388, 1055, 355, 524, 427, 1277, 1250,
                 1828, 1395, 23, 1088, 491, 46]
}
df = pd.DataFrame(data)

# Create a stacked bar chart (fig1)
fig1 = px.bar(df[df['decade'] != 'NaN'], 
              x='decade', 
              y='frequency', 
              color='fire_cause',
              title='BC Wildfires by Cause and Decade',
              labels={'decade': 'Decade', 
                     'frequency': 'Number of Fires',
                     'fire_cause': 'Cause'},
              barmode='stack')

# Create a pie chart for area burned (fig2)
area_data = {
    'fire_cause': ['Person', 'Lightning', 'Unknown'],
    'area_burned': [7.490765e+10, 1.255113e+11, 8.026890e+08]
}
area_df = pd.DataFrame(area_data)

fig2 = px.pie(area_df, 
              values='area_burned', 
              names='fire_cause',
              title='Distribution of Total Area Burned by Fire Cause',
              labels={'area_burned': 'Area Burned (sq meters)',
                     'fire_cause': 'Cause'})

# Create a pie chart for total proportions (fig3)
total_by_cause = df.groupby('fire_cause')['frequency'].sum()
fig3 = px.pie(values=total_by_cause.values, 
              names=total_by_cause.index,
              title='Total Distribution of BC Wildfires by Cause')

# Display all plots
fig1.show()
fig2.show()
fig3.show()

<div class="alert alert-block alert-info">
British Columbia's wildfire history tells a fascinating story through its causes and patterns. Lightning-caused fires, while representing only 35.72% of recorded incidents (8,570 fires), have been responsible for nearly two-thirds of all burned area. These natural fires average 14.65 million square meters each, with the largest reaching an extraordinary 7.69 billion square meters. In contrast, human-caused fires, though more numerous at 15,349 incidents (63.98%), typically burn smaller areas, averaging 4.88 million square meters.
The early decades show distinct human influence on fire patterns. The 1920s and 1930s experienced peak human-caused fire activity, with approximately 3,000 incidents per decade, alongside significant lightning-caused fires. This period of intensive land development laid the groundwork for modern fire management practices.
A stable pattern emerged from the 1940s through the 1980s, with human-caused fires consistently hovering around 1,000 incidents per decade. Lightning-caused fires during this period ranged from 355 to 530 incidents, though they often proved more destructive, particularly in remote northern regions.
The turn of the century marked a significant shift. The 2000s saw near-equal numbers of human and lightning-caused fires (1,277 and 1,250 respectively). This trend intensified in the 2010s, with lightning becoming the predominant cause - 1,828 lightning-fired fires compared to 1,395 human-caused incidents. Recent data from the 2020s shows this pattern continuing, with lightning strikes responsible for 1,088 fires compared to 491 human-caused incidents.
This shifting dynamic, coupled with factors like the Mountain Pine Beetle outbreak and changing climate conditions, has reshaped British Columbia's fire regime. The data reveals a clear transition from predominantly human-influenced fire patterns to one where natural causes increasingly drive both fire frequency and scale.
</div>

<div class="alert alert-block alert-success">
These next few steps demonstrate which seasons experience the most forest fires and whether there have been any changes over the years. One observation from the data is that more forest fires have occurred in the spring over the last few decades. Additionally, as predicted, forest fires are more common during the summer months and are more likely to spread during this time.
</div>

In [ ]:
%%sql
WITH fire_seasons AS (
    SELECT 
        CAST("FIRE_DATE" AS INTEGER) % 10000 as month_day,
        area_sqm,
        CASE 
            WHEN CAST("FIRE_DATE" AS INTEGER) % 10000 BETWEEN 301 AND 531 THEN 'Spring'   -- March 1 to May 31
            WHEN CAST("FIRE_DATE" AS INTEGER) % 10000 BETWEEN 601 AND 831 THEN 'Summer'   -- June 1 to August 31
            WHEN CAST("FIRE_DATE" AS INTEGER) % 10000 BETWEEN 901 AND 1130 THEN 'Fall'    -- September 1 to November 30
            ELSE 'Winter'                                      -- December 1 to February 29
        END as season
    FROM 'spatial.fires'
    WHERE CAST("FIRE_DATE" AS INTEGER) % 10000 > 0  -- Filter out invalid dates
),
totals AS (
    SELECT 
        COUNT(*) as total_fires,
        SUM(area_sqm) as total_area
    FROM fire_seasons
),
seasonal_stats AS (
    SELECT 
        season,
        COUNT(*) as num_fires,
        ROUND(AVG(area_sqm), 2) as avg_size,
        SUM(area_sqm) as total_area
    FROM fire_seasons
    GROUP BY season
)
SELECT 
    s.season,
    s.num_fires,
    s.avg_size,
    s.total_area,
    ROUND(CAST(s.num_fires AS FLOAT) / t.total_fires * 100, 2) as percentage_of_fires,
    ROUND(CAST(s.total_area AS FLOAT) / t.total_area * 100, 2) as percentage_of_area
FROM seasonal_stats s
CROSS JOIN totals t
ORDER BY CASE 
    WHEN season = 'Spring' THEN 1
    WHEN season = 'Summer' THEN 2
    WHEN season = 'Fall' THEN 3
    WHEN season = 'Winter' THEN 4
END;

In [ ]:
%%sql
WITH fire_decades AS (
    SELECT 
        CAST("FIRE_DATE" AS INTEGER) / 10000 as year,  -- Extract year from fire_date
        FLOOR(CAST("FIRE_DATE" AS INTEGER) / 100000) * 10 as decade,  -- Convert year to decade
        CAST("FIRE_DATE" AS INTEGER) % 10000 as month_day,
        area_sqm,
        CASE 
            WHEN CAST("FIRE_DATE" AS INTEGER) % 10000 BETWEEN 301 AND 531 THEN 'Spring'
            WHEN CAST("FIRE_DATE" AS INTEGER) % 10000 BETWEEN 601 AND 831 THEN 'Summer'
            WHEN CAST("FIRE_DATE" AS INTEGER) % 10000 BETWEEN 901 AND 1130 THEN 'Fall'
            ELSE 'Winter'
        END as season
    FROM 'spatial.fires'
    WHERE CAST("FIRE_DATE" AS INTEGER) % 10000 > 0  -- Filter out invalid dates
)
SELECT 
    decade || 's' as decade,
    season,
    COUNT(*) as num_fires,
    ROUND(AVG(area_sqm), 2) as avg_size,
    ROUND(SUM(area_sqm), 2) as total_area
FROM fire_decades
GROUP BY decade, season
ORDER BY 
    decade,
    CASE 
        WHEN season = 'Spring' THEN 1
        WHEN season = 'Summer' THEN 2
        WHEN season = 'Fall' THEN 3
        WHEN season = 'Winter' THEN 4
    END;

In [ ]:
seasonal_data = {
    'season': ['Spring', 'Summer', 'Fall', 'Winter'],
    'percentage_of_fires': [26.54, 61.79, 11.50, 0.18],
    'percentage_of_area': [22.18, 75.43, 2.37, 0.03]
}
seasonal_df = pd.DataFrame(seasonal_data)
# Create a pie chart for percentage of fires
fig1 = px.pie(seasonal_df, 
              values='percentage_of_fires', 
              names='season',
              title='Distribution of Fires by Season (%)')
# Create a pie chart for percentage of area
fig2 = px.pie(seasonal_df, 
              values='percentage_of_area', 
              names='season',
              title='Distribution of Burned Area by Season (%)')
# Create decade-season data for heatmap
decade_data = {
    'decade': ['1910s', '1910s', '1910s', '1920s', '1920s', '1920s', '1930s', '1930s', '1930s', '1930s',
               '1940s', '1940s', '1940s', '1950s', '1950s', '1950s', '1950s', '1960s', '1960s', '1960s',
               '1970s', '1970s', '1970s', '1970s', '1980s', '1980s', '1980s', '1980s', '1990s', '1990s',
               '1990s', '1990s', '2000s', '2000s', '2000s', '2000s', '2010s', '2010s', '2010s', '2010s',
               '2020s', '2020s', '2020s', '2020s'],
    'season': ['Spring', 'Summer', 'Fall', 'Spring', 'Summer', 'Fall', 'Spring', 'Summer', 'Fall', 'Winter',
               'Spring', 'Summer', 'Fall', 'Spring', 'Summer', 'Fall', 'Winter', 'Spring', 'Summer', 'Fall',
               'Spring', 'Summer', 'Fall', 'Winter', 'Spring', 'Summer', 'Fall', 'Winter', 'Spring', 'Summer',
               'Fall', 'Winter', 'Spring', 'Summer', 'Fall', 'Winter', 'Spring', 'Summer', 'Fall', 'Winter',
               'Spring', 'Summer', 'Fall', 'Winter'],
    'num_fires': [17, 207, 39, 820, 2724, 391, 931, 2631, 381, 3, 361, 1004, 118, 565, 786, 172, 5,
                  366, 991, 213, 349, 695, 457, 1, 458, 619, 330, 3, 232, 535, 174, 10, 1100, 1318,
                  107, 2, 851, 2132, 251, 12, 315, 1179, 125, 6]
}
decade_df = pd.DataFrame(decade_data)
# Create heatmap of fire distribution
fig3 = px.density_heatmap(decade_df,
                         x='decade',
                         y='season',
                         z='num_fires',
                         title='Fire Distribution by Decade and Season',
                         labels={'num_fires': 'Number of Fires'})
# Display all plots
fig1.show()
fig2.show()
fig3.show()

<div class="alert alert-block alert-info">
British Columbia's wildfire patterns follow a distinct seasonal rhythm, with data revealing compelling trends across different times of year. Summer emerges as the dominant fire season, accounting for 14,821 fires (61.79% of all incidents) and 75.43% of the total burned area. These summer fires average 10.24 million square meters in size, demonstrating their significant intensity.
Spring presents as the second most active period, with 6,365 recorded fires representing 26.54% of incidents and 22.18% of burned area. Spring fires average 7.01 million square meters, highlighting substantial early-season fire behavior.
The data shows a clear decline in fire activity during fall and winter months. Fall accounts for 2,758 fires (11.50% of total) but only 2.37% of burned area, with fires averaging 1.73 million square meters. Winter fires are exceptionally rare, with just 42 recorded incidents averaging 1.27 million square meters each.
Recent decades reveal evolving patterns. While summer traditionally dominated fire activity - with notable peaks of over 2,600 fires in the 1920s and 1930s - the 2020s show a striking shift in fire intensity. Spring fires in this period average an unprecedented 48.43 million square meters, while summer fires average 26.73 million square meters. This trend suggests that although fire frequency might not have dramatically increased, the scale and intensity of fires have grown significantly.
This seasonal analysis provides crucial insights for fire management strategies, particularly as changing climate conditions continue to influence fire behavior patterns across British Columbia.
</div>

In [ ]:
%%sql -o fires_data
SELECT 
    objectid,
    CAST("FIRE_DATE" AS INTEGER) as fire_date,
    area_sqm,
    fire_cause,
    geometry
FROM 'spatial.fires'
LIMIT 5000;

In [ ]:
import json
from shapely.geometry import shape

# Convert the geometry column from JSON strings to Shapely geometries
fires_data['geometry'] = fires_data['geometry'].apply(lambda geom: shape(json.loads(geom)) if geom else None)

# Verify the geometry column
print(fires_data['geometry'].head())

In [ ]:
import geopandas as gpd

# Convert to GeoDataFrame with the correct CRS (EPSG:26910)
gdf = gpd.GeoDataFrame(
    fires_data,
    geometry='geometry',  # The geometry column is now a Shapely object
    crs="EPSG:26910"  # NAD 1983 UTM Zone 10N
)

# Reproject to WGS84 (required for web mapping)
gdf = gdf.to_crs("EPSG:4326")

# Verify the GeoDataFrame
print(gdf.head())

In [ ]:
# Extract the year from the fire_date (YYYYMMDD format)
gdf['year'] = gdf['fire_date'] // 10000

# Calculate the decade
gdf['decade'] = (gdf['year'] // 10) * 10

# Verify the decades
print("Unique decades:")
print(gdf['decade'].unique())

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Define a function to filter data by decade
def filter_by_decade(decade):
    filtered_gdf = gdf[gdf['decade'] == decade]
    print(f"Number of fires in the {decade}s: {len(filtered_gdf)}")
    return filtered_gdf

# Create a slider for selecting the decade
decade_slider = widgets.IntSlider(
    value=1920,  # Default value
    min=gdf['decade'].min(),  # Minimum decade
    max=gdf['decade'].max(),  # Maximum decade
    step=10,  # Step size (10 years)
    description='Decade:',
    continuous_update=False  # Update only when the slider is released
)

# Display the slider
display(decade_slider)

<div class="alert alert-block alert-success">
Just as a reminder for the next part to work with a mapbox key is need.
</div>

In [ ]:
from IPython.display import HTML, display, Javascript, display_html
import os

# Retrieve the Mapbox token from the environment variable
mapbox_token = os.getenv('MAPBOX_ACCESS_TOKEN')
if not mapbox_token:
    raise ValueError("Please set the MAPBOX_ACCESS_TOKEN environment variable.")

# Define the custom legend HTML (static part)
custom_legend = """
<div id='custom-legend' style='
    position: absolute;
    bottom: 50px;
    right: 10px;
    background: white;
    padding: 10px;
    border-radius: 3px;
    box-shadow: 0 1px 2px rgba(0,0,0,0.1);
'>
    <h4 style='margin: 0 0 10px 0;'>Fire Causes</h4>
    <div>
        <span style='display: inline-block; width: 12px; height: 12px; background: #FF0000; margin-right: 5px;'></span>
        <span>Person</span>
    </div>
    <div style='margin-top: 5px;'>
        <span style='display: inline-block; width: 12px; height: 12px; background: #FFA500; margin-right: 5px;'></span>
        <span>Lightning</span>
    </div>
    <div style='margin-top: 5px;'>
        <span style='display: inline-block; width: 12px; height: 12px; background: #808080; margin-right: 5px;'></span>
        <span>Unknown</span>
    </div>
</div>
"""

# Base HTML template for the iframe (static part)
base_html = f"""
<!DOCTYPE html>
<html>
<head>
    <meta charset='utf-8' />
    <title>Fire Causes Map</title>
    <meta name='viewport' content='width=device-width, initial-scale=1' />
    <script src='https://api.mapbox.com/mapbox-gl-js/v2.10.0/mapbox-gl.js'></script>
    <link href='https://api.mapbox.com/mapbox-gl-js/v2.10.0/mapbox-gl.css' rel='stylesheet' />
    <style>
        body {{ margin: 0; padding: 0; }}
        #map {{ position: absolute; top: 0; bottom: 0; width: 100%; }}
        .legend {{ display: none; }}
    </style>
</head>
<body>
    <div id='map'></div>
    {custom_legend}
    <script>
        mapboxgl.accessToken = '{mapbox_token}';
        var map = new mapboxgl.Map({{
            container: 'map',
            style: 'mapbox://styles/mapbox/streets-v11',
            center: [-123.3656, 54.5362],
            zoom: 5
        }});

        map.on('load', function() {{
            map.addSource('fire-data', {{
                type: 'geojson',
                data: {{
                    type: 'FeatureCollection',
                    features: []
                }}
            }});

            map.addLayer({{
                id: 'fire-layer',
                type: 'fill',
                source: 'fire-data',
                paint: {{
                    'fill-color': [
                        'match',
                        ['get', 'fire_cause_numeric'],
                        1, '#FF0000',
                        2, '#FFA500',
                        3, '#808080',
                        'rgba(255,255,255,0)'
                    ],
                    'fill-opacity': 0.7
                }}
            }});
        }});

        function updateMapData(geojsonData) {{
            if (map.getSource('fire-data')) {{
                map.getSource('fire-data').setData(geojsonData);
            }}
        }}
    </script>
</body>
</html>
"""

# Create a single output widget for updates
from IPython.display import display, Javascript
import ipywidgets as widgets

update_output = widgets.Output()
display(update_output)

# Display the iframe with the base map
iframe = HTML(f"""
<iframe id="map-iframe" srcdoc="{base_html}" width="1400" height="800" style="border: none;"></iframe>
""")
display(iframe)

# Function to update the map data
def update_map(decade):
    # Filter the data by the selected decade
    filtered_gdf = gdf[gdf['decade'] == decade]
    
    # Convert the filtered GeoDataFrame to GeoJSON
    geojson_data = json.loads(filtered_gdf.to_json())
    
    # Add numeric mapping for fire causes
    fire_cause_mapping = {'Person': 1, 'Lightning': 2, 'Unknown': 3}
    for feature in geojson_data['features']:
        cause = feature['properties']['fire_cause']
        feature['properties']['fire_cause_numeric'] = fire_cause_mapping.get(cause, 0)
    
    # Clear previous output and update the map
    with update_output:
        update_output.clear_output(wait=True)
        display(Javascript(f"""
            var iframe = document.getElementById('map-iframe');
            if (iframe && iframe.contentWindow) {{
                iframe.contentWindow.updateMapData({json.dumps(geojson_data)});
            }}
        """))

# Create a slider for the decade selection
decade_slider = widgets.IntSlider(
    value=1910,
    min=1910,
    max=2020,
    step=10,
    description='Decade:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

# Use interact to link the slider to the update_map function
widgets.interact(update_map, decade=decade_slider);

In [ ]:
%%sql -o current_fires
SELECT 
    objectid,
    start_date,
    load_date,
    fire_sz_ha,
    fire_stat,
    fire_link,
    geometry
FROM 'bc.gov.current.fires'
LIMIT 500;

In [ ]:
# Convert the geometry column from JSON strings to Shapely geometries
current_fires['geometry'] = current_fires['geometry'].apply(lambda geom: shape(json.loads(geom)) if geom else None)

# Verify the geometry column
print(current_fires['geometry'].head())

In [ ]:
# Convert to GeoDataFrame with the correct CRS (EPSG:3153)
gdf = gpd.GeoDataFrame(
    current_fires,
    geometry='geometry',  # The geometry column is now a Shapely object
    crs="EPSG:3153"  # NAD 1983 BC Environment Albers
)
# Reproject to WGS84 (required for web mapping)
gdf = gdf.to_crs("EPSG:4326")
# Verify the GeoDataFrame
print(gdf.head())

In [ ]:
import folium
from datetime import datetime

# Function to convert columns to appropriate types for JSON serialization
def convert_to_json_serializable(gdf):
    # Make a copy to avoid modifying the original
    df_copy = gdf.copy()
    
    # Convert timestamps and dates to strings
    for col in df_copy.columns:
        if pd.api.types.is_datetime64_any_dtype(df_copy[col]):
            df_copy[col] = df_copy[col].astype(str)
    
    return df_copy

# Create a simple folium map solution
def create_folium_map():
    print("Creating forest fire map...")
    
    # Convert the GeoDataFrame
    gdf = gpd.GeoDataFrame(
        current_fires,
        geometry='geometry',
        crs="EPSG:3153"
    )
    gdf = gdf.to_crs("EPSG:4326")
    
    # Print sample coordinates
    print(f"Coordinate sample: {gdf.iloc[0].geometry.centroid.x}, {gdf.iloc[0].geometry.centroid.y}")
    
    # Convert timestamp fields to strings to avoid JSON serialization error
    gdf_converted = convert_to_json_serializable(gdf)
    
    # Create a manual GeoJSON structure instead of using to_json()
    features = []
    for idx, row in gdf_converted.iterrows():
        properties = {col: row[col] for col in gdf_converted.columns if col != 'geometry'}
        feature = {
            "type": "Feature",
            "properties": properties,
            "geometry": row['geometry'].__geo_interface__
        }
        features.append(feature)
    
    geojson_data = {
        "type": "FeatureCollection",
        "features": features
    }
    
    print(f"Successfully created GeoJSON with {len(features)} features")
    
    # Create a Folium map
    m = folium.Map(location=[54.5362, -123.3656], zoom_start=5)
    
    # Define a style function for the features
    def style_function(feature):
        fire_stat = feature['properties'].get('fire_stat', 'NULL')
        if fire_stat == 'Out':
            color = '#00FF00'  # Green
        elif fire_stat == 'Under Control':
            color = '#0000FF'  # Blue
        else:
            color = '#FF0000'  # Red for NULL or any other
        
        return {
            'fillColor': color,
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.7
        }
    
    # Create a tooltip
    tooltip_fields = ['objectid', 'fire_stat', 'fire_sz_ha', 'start_date']
    
    # Make sure all fields exist
    tooltip_fields = [field for field in tooltip_fields if field in gdf_converted.columns]
    
    tooltip = folium.GeoJsonTooltip(
        fields=tooltip_fields,
        aliases=['ID', 'Status', 'Size (ha)', 'Start Date'],
        localize=True
    )
    
    # Add the data to the map
    folium.GeoJson(
        geojson_data,
        name='BC Forest Fires',
        style_function=style_function,
        tooltip=tooltip
    ).add_to(m)
    
    # Add a legend
    legend_html = '''
    <div style="position: fixed; 
                bottom: 50px; right: 50px; 
                border:2px solid grey; z-index:9999; 
                background-color:white;
                padding: 10px;
                border-radius: 5px;
                ">
      <h4 style="margin-top: 0; margin-bottom: 10px;">Fire Status</h4>
      <div style="display: flex; align-items: center; margin-bottom: 5px;">
        <div style="background-color: #00FF00; width: 20px; height: 20px; margin-right: 5px;"></div>
        <span>Out</span>
      </div>
      <div style="display: flex; align-items: center; margin-bottom: 5px;">
        <div style="background-color: #0000FF; width: 20px; height: 20px; margin-right: 5px;"></div>
        <span>Under Control</span>
      </div>
      <div style="display: flex; align-items: center;">
        <div style="background-color: #FF0000; width: 20px; height: 20px; margin-right: 5px;"></div>
        <span>NULL/Unknown</span>
      </div>
    </div>
    '''
    m.get_root().html.add_child(folium.Element(legend_html))
    
    # Return the map
    return m

# Create and display the map
fire_map = create_folium_map()
display(fire_map)

In [ ]:
import plotly.express as px
import pandas as pd

# Convert start_date to datetime and extract the month
current_fires['start_date'] = pd.to_datetime(current_fires['start_date'])
current_fires['month'] = current_fires['start_date'].dt.month

# Map month numbers to month names
month_names = {
    1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June',
    7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'
}
current_fires['month_name'] = current_fires['month'].map(month_names)

# Count the number of fires per month
monthly_fires = current_fires['month_name'].value_counts().sort_index().reset_index()
monthly_fires.columns = ['month_name', 'count']

# Sort the months in chronological order
monthly_fires['month_name'] = pd.Categorical(
    monthly_fires['month_name'],
    categories=list(month_names.values()),
    ordered=True
)
monthly_fires = monthly_fires.sort_values('month_name')

# Create a bar chart with a taller height
bar_fig = px.bar(
    monthly_fires,
    x='month_name',
    y='count',
    labels={'month_name': 'Month', 'count': 'Number of Fires'},
    title='Number of Fires Started by Month',
    text='count',  # Add the count as text above the bars
    height=600  # Increase the height of the graph
)

# Update layout for better readability
bar_fig.update_layout(
    xaxis=dict(
        title='Month',
        tickmode='array',
        tickvals=list(range(12))
    ),
    yaxis=dict(title='Number of Fires'),
    showlegend=False,
    margin=dict(t=50)  # Add margin at the top to prevent text cutoff
)

# Customize the text position and appearance
bar_fig.update_traces(textposition='outside', textfont_size=12)

# Display the bar chart
bar_fig.show()

In [ ]:
# Categorize fire status into 'Out', 'Under Control', and 'Other'
current_fires['fire_stat_category'] = current_fires['fire_stat'].apply(
    lambda x: 'Out' if x == 'Out' else ('Under Control' if x == 'Under Control' else 'Other')
)

# Count the number of fires in each category
fire_status_counts = current_fires['fire_stat_category'].value_counts().reset_index()
fire_status_counts.columns = ['status', 'count']

# Create a pie chart
pie_fig = px.pie(
    fire_status_counts,
    names='status',
    values='count',
    title='Fire Status Distribution'
)

# Display the pie chart
pie_fig.show()